# Parameter search with optuna - ETNN

## Importing section

In [1]:
import optuna

import numpy as np

import torch
from sklearn.metrics import r2_score
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.routines.run_config import choice_dataset, choice_trainloader, choice_loss, choice_optim
from etnn.tools.training import train_epoch, eval_epoch
from etnn.tools.training_tools import ConfigStore, seeding_all

## Definition of objective function for ETNN

In [2]:
def objective(trial):
    # init default config
    config = ConfigStore(
        in_dim=15,
        hidden_dim=trial.suggest_int("hidden_dim", 16, 512, step=16),
        out_dim=1,
        k=trial.suggest_int("k", 1, 5),
        dataset=-2,
        ds_size=dataset_size,
        num_gondolas=10,
        num_part_pg=5,
        loss_name='mse',
        optimizer_name='adam',
        num_max_epochs=30, # real: 100
        learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        batch_size=1024,
        early_stop_tol=5,
        use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
        seed=420,
        label_type=label,
        final_label_factor=1/1000
    )

    # loading dataset
    dataset, df_index = choice_dataset(config, dataset_path)
    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # build tree
    tree_structure = TreeNode(
        node_type="C",
        children=[
            TreeNode("P", [TreeNode("E", config.num_part_pg)])
            for _ in range(config.num_gondolas)
        ]
    )

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=tree_structure,
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## DS size 10

In [3]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50
dataset_size = 10

In [4]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2024-05-25 11:29:10,707] A new study created in memory with name: Best default label config


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:00<?, ?it/s][A

100%|██████████| 10/10 [00:00<00:00, 103.40it/s]


[I 2024-05-25 11:29:14,634] Trial 0 finished with value: -2.680610472404536 and parameters: {'hidden_dim': 128, 'k': 3, 'learning_rate': 0.0007297245197178391, 'batcher': True}. Best is trial 0 with value: -2.680610472404536.
[I 2024-05-25 11:29:15,177] Trial 1 finished with value: -266.9170577248465 and parameters: {'hidden_dim': 352, 'k': 2, 'learning_rate': 1.1536985000251613e-05, 'batcher': True}. Best is trial 0 with value: -2.680610472404536.
[I 2024-05-25 11:29:15,645] Trial 2 finished with value: -2.8488131946788386 and parameters: {'hidden_dim': 112, 'k': 2, 'learning_rate': 0.0009184716920136767, 'batcher': True}. Best is trial 0 with value: -2.680610472404536.
[I 2024-05-25 11:29:16,124] Trial 3 finished with value: -3.642423898190845 and parameters: {'hidden_dim': 304, 'k': 1, 'learning_rate': 0.00018486519701524216, 'batcher': True}. Best is trial 0 with value: -2.680610472404536.
[I 2024-05-25 11:29:16,650] Trial 4 finished with value: -130859.29755165361 and parameters: 

In [6]:
df_default = study_default.trials_dataframe()
df_default.to_csv(f"study_label-default_post-normalized_ds-{dataset_size}.csv")
df_default

,number,value,datetime_start,datetime_complete,duration,params_batcher,params_hidden_dim,params_k,params_learning_rate,state
0,0,-2.680610e+00,2024-05-25 11:29:10.715062,2024-05-25 11:29:14.634671,0 days 00:00:03.919609,True,128,3,0.000730,COMPLETE
1,1,-2.669171e+02,2024-05-25 11:29:14.635671,2024-05-25 11:29:15.177720,0 days 00:00:00.542049,True,352,2,0.000012,COMPLETE
2,2,-2.848813e+00,2024-05-25 11:29:15.178721,2024-05-25 11:29:15.645694,0 days 00:00:00.466973,True,112,2,0.000918,COMPLETE
3,3,-3.642424e+00,2024-05-25 11:29:15.646699,2024-05-25 11:29:16.124481,0 days 00:00:00.477782,True,304,1,0.000185,COMPLETE
4,4,-1.308593e+05,2024-05-25 11:29:16.125481,2024-05-25 11:29:16.650980,0 days 00:00:00.525499,True,512,2,0.008041,COMPLETE
5,5,-4.406165e+02,2024-05-25 11:29:16.651980,2024-05-25 11:29:17.138697,0 days 00:00:00.486717,False,416,1,0.008622,COMPLETE
6,6,-1.892555e+02,2024-05-25 11:29:17.139696,2024-05-25 11:29:17.692280,0 days 00:00:00.552584,True,192,4,0.000092,COMPLETE
7,7,-1.009566e+01,2024-05-25 11:29:17.693280,2024-05-25 11:29:18.187552,0 days 00:00:00.494272,True,432,2,0.000128,COMPLETE
8,8,-2.810715e+01,2024-05-25 11:29:18.188552,2024-05-25 11:29:18.702662,0 days 00:00:00.514110,True,224,3,0.000124,COMPLETE
9,9,-6.832694e+02,2024-05-25 11:29:18.703662,2024-05-25 11:29:19.270426,0 days 00:00:00.566764,False,176,5,0.007893,COMPLETE


## DS size 100

In [7]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50
dataset_size = 100

In [8]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2024-05-25 11:30:46,019] A new study created in memory with name: Best default label config


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:00<?, ?it/s]A

100%|██████████| 100/100 [00:03<00:00, 26.97it/s][A


[I 2024-05-25 11:30:51,573] Trial 0 finished with value: -5.785649199399307 and parameters: {'hidden_dim': 48, 'k': 3, 'learning_rate': 0.0017410333511225565, 'batcher': True}. Best is trial 0 with value: -5.785649199399307.
[I 2024-05-25 11:30:53,012] Trial 1 finished with value: -58.13116903260546 and parameters: {'hidden_dim': 480, 'k': 2, 'learning_rate': 2.9188344056962752e-05, 'batcher': False}. Best is trial 0 with value: -5.785649199399307.
[I 2024-05-25 11:30:54,231] Trial 2 finished with value: -66.4754801404125 and parameters: {'hidden_dim': 64, 'k': 3, 'learning_rate': 0.00016673376939950972, 'batcher': False}. Best is trial 0 with value: -5.785649199399307.
[I 2024-05-25 11:30:55,561] Trial 3 finished with value: -78.88594637206013 and parameters: {'hidden_dim': 240, 'k': 4, 'learning_rate': 3.464987433796385e-05, 'batcher': False}. Best is trial 0 with value: -5.785649199399307.
[I 2024-05-25 11:30:56,979] Trial 4 finished with value: -1.3164739710714717 and parameters: {

In [9]:
df_default = study_default.trials_dataframe()
df_default.to_csv(f"study_label-default_post-normalized_ds-{dataset_size}.csv")
df_default

,number,value,datetime_start,datetime_complete,duration,params_batcher,params_hidden_dim,params_k,params_learning_rate,state
0,0,-5.785649,2024-05-25 11:30:46.025177,2024-05-25 11:30:51.573545,0 days 00:00:05.548368,True,48,3,0.001741,COMPLETE
1,1,-58.131169,2024-05-25 11:30:51.575546,2024-05-25 11:30:53.012075,0 days 00:00:01.436529,False,480,2,0.000029,COMPLETE
2,2,-66.475480,2024-05-25 11:30:53.013075,2024-05-25 11:30:54.230531,0 days 00:00:01.217456,False,64,3,0.000167,COMPLETE
3,3,-78.885946,2024-05-25 11:30:54.232531,2024-05-25 11:30:55.561902,0 days 00:00:01.329371,False,240,4,0.000035,COMPLETE
4,4,-1.316474,2024-05-25 11:30:55.562902,2024-05-25 11:30:56.979224,0 days 00:00:01.416322,True,240,5,0.000602,COMPLETE
5,5,-83.610510,2024-05-25 11:30:56.981224,2024-05-25 11:30:58.471592,0 days 00:00:01.490368,True,272,5,0.000024,COMPLETE
6,6,-0.544952,2024-05-25 11:30:58.473590,2024-05-25 11:30:59.863920,0 days 00:00:01.390330,True,304,2,0.002645,COMPLETE
7,7,-88.529602,2024-05-25 11:30:59.864920,2024-05-25 11:31:01.354922,0 days 00:00:01.490002,False,416,2,0.000012,COMPLETE
8,8,-93.734012,2024-05-25 11:31:01.356921,2024-05-25 11:31:02.549045,0 days 00:00:01.192124,False,16,1,0.000724,COMPLETE
9,9,-0.208900,2024-05-25 11:31:02.550046,2024-05-25 11:31:04.192343,0 days 00:00:01.642297,False,432,5,0.001137,COMPLETE


## DS size 1000

In [10]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50
dataset_size = 1000

In [11]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2024-05-25 11:32:01,837] A new study created in memory with name: Best default label config


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1000/1000 [00:00<00:00, 111125.05it/s]

100%|██████████| 1000/1000 [00:05<00:00, 171.28it/s][A


[I 2024-05-25 11:32:18,486] Trial 0 finished with value: -109.95837044351859 and parameters: {'hidden_dim': 176, 'k': 4, 'learning_rate': 1.98873001507753e-05, 'batcher': True}. Best is trial 0 with value: -109.95837044351859.
[I 2024-05-25 11:32:29,095] Trial 1 finished with value: -3.3945428567978397 and parameters: {'hidden_dim': 432, 'k': 3, 'learning_rate': 0.004419176259406183, 'batcher': True}. Best is trial 1 with value: -3.3945428567978397.
[I 2024-05-25 11:32:39,124] Trial 2 finished with value: -0.7043404076849129 and parameters: {'hidden_dim': 144, 'k': 5, 'learning_rate': 0.005462754015951113, 'batcher': True}. Best is trial 2 with value: -0.7043404076849129.
[I 2024-05-25 11:32:49,781] Trial 3 finished with value: -1.0952784220711504 and parameters: {'hidden_dim': 416, 'k': 5, 'learning_rate': 8.822373473108787e-05, 'batcher': False}. Best is trial 2 with value: -0.7043404076849129.
[I 2024-05-25 11:33:00,233] Trial 4 finished with value: -1.2933203922912697 and parameter

In [12]:
df_default = study_default.trials_dataframe()
df_default.to_csv(f"study_label-default_post-normalized_ds-{dataset_size}.csv")
df_default

,number,value,datetime_start,datetime_complete,duration,params_batcher,params_hidden_dim,params_k,params_learning_rate,state
0,0,-1.099584e+02,2024-05-25 11:32:01.842273,2024-05-25 11:32:18.486421,0 days 00:00:16.644148,True,176,4,0.000020,COMPLETE
1,1,-3.394543e+00,2024-05-25 11:32:18.487421,2024-05-25 11:32:29.094227,0 days 00:00:10.606806,True,432,3,0.004419,COMPLETE
2,2,-7.043404e-01,2024-05-25 11:32:29.096227,2024-05-25 11:32:39.124867,0 days 00:00:10.028640,True,144,5,0.005463,COMPLETE
3,3,-1.095278e+00,2024-05-25 11:32:39.125866,2024-05-25 11:32:49.781983,0 days 00:00:10.656117,False,416,5,0.000088,COMPLETE
4,4,-1.293320e+00,2024-05-25 11:32:49.782983,2024-05-25 11:33:00.233532,0 days 00:00:10.450549,True,304,2,0.003457,COMPLETE
5,5,-9.936028e+01,2024-05-25 11:33:00.234533,2024-05-25 11:33:11.113880,0 days 00:00:10.879347,True,480,4,0.006639,COMPLETE
6,6,-5.433723e+00,2024-05-25 11:33:11.115883,2024-05-25 11:33:20.251531,0 days 00:00:09.135648,True,32,2,0.003861,COMPLETE
7,7,-7.120106e-01,2024-05-25 11:33:20.253532,2024-05-25 11:33:31.107522,0 days 00:00:10.853990,False,512,1,0.002642,COMPLETE
8,8,-1.036012e+02,2024-05-25 11:33:31.109523,2024-05-25 11:33:41.024329,0 days 00:00:09.914806,True,80,5,0.000106,COMPLETE
9,9,-6.342816e-01,2024-05-25 11:33:41.025328,2024-05-25 11:33:50.955201,0 days 00:00:09.929873,True,64,1,0.005497,COMPLETE


## DS size 10000

In [13]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50
dataset_size = 10000

In [14]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2024-05-25 11:41:20,923] A new study created in memory with name: Best default label config


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-05-25 11:43:02,808] Trial 0 finished with value: -0.01432071980946743 and parameters: {'hidden_dim': 512, 'k': 3, 'learning_rate': 0.002544013355403776, 'batcher': False}. Best is trial 0 with value: -0.01432071980946743.
[I 2024-05-25 11:44:39,931] Trial 1 finished with value: 0.00024775423375580365 and parameters: {'hidden_dim': 288, 'k': 4, 'learning_rate': 0.0004614842074493659, 'batcher': True}. Best is trial 1 with value: 0.00024775423375580365.
[I 2024-05-25 11:46:16,739] Trial 2 finished with value: -0.34343452012273573 and parameters: {'hidden_dim': 192, 'k': 1, 'learning_rate': 0.00019029758198167105, 'batcher': True}. Best is trial 1 with value: 0.00024775423375580365.
[I 2024-05-25 11:47:58,306] Trial 3 finished with value: -0.06010592644675441 and parameters: {'hidden_dim': 480, 'k': 1, 'learning_rate': 0.00012922091687107856, 'batcher': False}. Best is trial 1 with value: 0.00024775423375580365.
[I 2024-05-25 11:49:31,344] Trial 4 finished with value: -108.2398044

In [15]:
df_default = study_default.trials_dataframe()
df_default.to_csv(f"study_label-default_post-normalized_ds-{dataset_size}.csv")
df_default

,number,value,datetime_start,datetime_complete,duration,params_batcher,params_hidden_dim,params_k,params_learning_rate,state
0,0,-0.014321,2024-05-25 11:41:20.930522,2024-05-25 11:43:02.808818,0 days 00:01:41.878296,False,512,3,0.002544,COMPLETE
1,1,0.000248,2024-05-25 11:43:02.809818,2024-05-25 11:44:39.931133,0 days 00:01:37.121315,True,288,4,0.000461,COMPLETE
2,2,-0.343435,2024-05-25 11:44:39.932133,2024-05-25 11:46:16.739425,0 days 00:01:36.807292,True,192,1,0.000190,COMPLETE
3,3,-0.060106,2024-05-25 11:46:16.740424,2024-05-25 11:47:58.306642,0 days 00:01:41.566218,False,480,1,0.000129,COMPLETE
4,4,-108.239804,2024-05-25 11:47:58.307641,2024-05-25 11:49:31.343581,0 days 00:01:33.035940,False,192,2,0.000015,COMPLETE
5,5,0.085612,2024-05-25 11:49:31.344581,2024-05-25 11:51:15.447532,0 days 00:01:44.102951,False,256,1,0.000699,COMPLETE
6,6,-0.397630,2024-05-25 11:51:15.448501,2024-05-25 11:53:04.092735,0 days 00:01:48.644234,True,352,2,0.000052,COMPLETE
7,7,-0.220045,2024-05-25 11:53:04.093735,2024-05-25 11:54:57.111771,0 days 00:01:53.018036,False,192,5,0.000088,COMPLETE
8,8,-130.867886,2024-05-25 11:54:57.113772,2024-05-25 11:56:38.148934,0 days 00:01:41.035162,True,32,4,0.000015,COMPLETE
9,9,-0.305836,2024-05-25 11:56:38.149934,2024-05-25 11:58:28.758230,0 days 00:01:50.608296,False,336,2,0.000022,COMPLETE


## DS size 100000

In [3]:
# setting global parameters
dataset_path = "../../datasets/"
label = "default" # alt: tree or default
test_perc = 0.3
val_perc = 0.21
stability_count = 5
n_trials = 50
dataset_size = 100000

In [4]:
study_default = optuna.create_study(study_name="Best default label config", directions=['maximize'])
study_default.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2024-05-25 14:33:25,751] A new study created in memory with name: Best default label config


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 100000/100000 [00:00<00:00, 154311.68it/s][A

100%|██████████| 100000/100000 [04:21<00:00, 382.45it/s][A


[I 2024-05-25 14:56:56,911] Trial 0 finished with value: 0.3537597959653131 and parameters: {'hidden_dim': 432, 'k': 5, 'learning_rate': 0.006433218832637501, 'batcher': False}. Best is trial 0 with value: 0.3537597959653131.
[I 2024-05-25 15:15:16,963] Trial 1 finished with value: 0.4083797786356455 and parameters: {'hidden_dim': 416, 'k': 5, 'learning_rate': 0.00011858500393507517, 'batcher': True}. Best is trial 1 with value: 0.4083797786356455.
[I 2024-05-25 15:33:57,417] Trial 2 finished with value: 0.188933301910993 and parameters: {'hidden_dim': 320, 'k': 4, 'learning_rate': 0.007423269190667876, 'batcher': True}. Best is trial 1 with value: 0.4083797786356455.
[I 2024-05-25 15:54:41,557] Trial 3 finished with value: 0.45550190346570724 and parameters: {'hidden_dim': 112, 'k': 3, 'learning_rate': 0.00045896036229507394, 'batcher': False}. Best is trial 3 with value: 0.45550190346570724.
[I 2024-05-25 16:15:37,340] Trial 4 finished with value: 0.6216939775870522 and parameters: {

In [5]:
df_default = study_default.trials_dataframe()
df_default.to_csv(f"study_label-default_post-normalized_ds-{dataset_size}.csv")
df_default

,number,value,datetime_start,datetime_complete,duration,params_batcher,params_hidden_dim,params_k,params_learning_rate,state
0,0,0.353760,2024-05-25 14:33:25.758669,2024-05-25 14:56:56.911838,0 days 00:23:31.153169,False,432,5,0.006433,COMPLETE
1,1,0.408380,2024-05-25 14:56:56.913838,2024-05-25 15:15:16.963460,0 days 00:18:20.049622,True,416,5,0.000119,COMPLETE
2,2,0.188933,2024-05-25 15:15:16.964459,2024-05-25 15:33:57.416062,0 days 00:18:40.451603,True,320,4,0.007423,COMPLETE
3,3,0.455502,2024-05-25 15:33:57.418295,2024-05-25 15:54:41.557896,0 days 00:20:44.139601,False,112,3,0.000459,COMPLETE
4,4,0.621694,2024-05-25 15:54:41.559896,2024-05-25 16:15:37.340422,0 days 00:20:55.780526,True,80,3,0.004315,COMPLETE
5,5,-96.016562,2024-05-25 16:15:37.342423,2024-05-25 16:36:07.114482,0 days 00:20:29.772059,True,16,3,0.006919,COMPLETE
6,6,-0.625448,2024-05-25 16:36:07.115481,2024-05-25 16:53:49.861071,0 days 00:17:42.745590,True,32,4,0.000069,COMPLETE
7,7,0.756061,2024-05-25 16:53:49.862073,2024-05-25 17:13:07.296075,0 days 00:19:17.434002,False,128,5,0.007614,COMPLETE
8,8,-0.250425,2024-05-25 17:13:07.299078,2024-05-25 17:34:29.268719,0 days 00:21:21.969641,False,96,2,0.000014,COMPLETE
9,9,-0.005554,2024-05-25 17:34:29.270719,2024-05-25 17:56:15.190418,0 days 00:21:45.919699,False,224,4,0.000019,COMPLETE
